In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV

from collections import Counter

In [6]:
df = pd.read_csv("D:\final-behnaz\final\chapters\code\churn_prediction.csv")
df.shape

FileNotFoundError: [Errno 2] No such file or directory: 'D:\x0cinal-behnaz\x0cinal\\chapters\\code\\churn_prediction.csv'

In [ ]:
df['churn'].value_counts()

## Data Preprocessing

In [ ]:
df.isnull().sum()

### `a) Missing Value Treatment`

In [ ]:
#Convert Gender
dict_gender = {'Male': 1, 'Female':0}
df.replace({'gender': dict_gender}, inplace = True)

# Replace with -1 for missing gender
df['gender'] = df['gender'].fillna(-1)

# Replacing with max. occurence values
df['dependents'] = df['dependents'].fillna(0)
df['occupation'] = df['occupation'].fillna('self_employed')
df['city'] = df['city'].fillna(1020)

### `b) Dummy variables`

In [ ]:
# Convert occupation to one hot encoded features
df = pd.concat([df,pd.get_dummies(df['occupation'],prefix = str('occupation'),prefix_sep='_')],axis = 1)

In [ ]:
df.head()

# Train and Test Split

In [ ]:
#x = df.drop('Attrition', axis=1)
x = df.drop(['churn','customer_id', 'occupation', 'last_transaction'], axis=1)
y = df['churn']
# Splitting the data into train and test
X_train,X_test,y_train,y_test=train_test_split(x, y, train_size=0.8, stratify = y, random_state=100)

In [ ]:
y_train.shape, y_test.shape

In [ ]:
y_train.value_counts()/len(y_train)

In [ ]:
y_test.value_counts()/len(y_test)

In [ ]:
from sklearn.preprocessing import StandardScaler
Scaler_X = StandardScaler()
X_train = Scaler_X.fit_transform(X_train)
X_test = Scaler_X.transform(X_test)

# Handling class imbalance using SMOTE based techniques

### A) SMOTE Technique

In [ ]:
from imblearn.over_sampling import SMOTE

counter = Counter(y_train)
print('Before',counter)
# oversampling the train dataset using SMOTE
smt = SMOTE()
#X_train, y_train = smt.fit_resample(X_train, y_train)
X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

counter = Counter(y_train_sm)
print('After',counter)

### B) ADASYN Technique

In [ ]:
from imblearn.over_sampling import ADASYN

counter = Counter(y_train)
print('Before',counter)
# oversampling the train dataset using ADASYN
ada = ADASYN(random_state=130)
X_train_ada, y_train_ada = ada.fit_resample(X_train, y_train)

counter = Counter(y_train_ada)
print('After',counter)


## C) Hybrid Techniques

### C.1) SMOTE + Tomek Links

In [ ]:
from imblearn.combine import SMOTETomek

counter = Counter(y_train)
print('Before',counter)
# oversampling the train dataset using SMOTE + Tomek
smtom = SMOTETomek(random_state=139)
X_train_smtom, y_train_smtom = smtom.fit_resample(X_train, y_train)

counter = Counter(y_train_smtom)
print('After',counter)


### C.2) SMOTE + ENN

In [ ]:
from imblearn.combine import SMOTEENN

counter = Counter(y_train)
print('Before',counter)
# oversampling the train dataset using SMOTE + ENN
smenn = SMOTEENN()
X_train_smenn, y_train_smenn = smenn.fit_resample(X_train, y_train)

counter = Counter(y_train_smenn)
print('After',counter)

# Model Building - Imbalanced data

In [ ]:
model = list()
resample = list()
precision = list()
recall = list()
F1score = list()
AUCROC = list()

In [ ]:
def test_eval(clf_model, X_test, y_test, algo=None, sampling=None):
    # Test set prediction
    y_prob=clf_model.predict_proba(X_test)
    y_pred=clf_model.predict(X_test)

    print('Confusion Matrix')
    print('='*60)
    print(confusion_matrix(y_test,y_pred),"\n")
    print('Classification Report')
    print('='*60)
    print(classification_report(y_test,y_pred),"\n")
    print('AUC-ROC')
    print('='*60)
    print(roc_auc_score(y_test, y_prob[:,1]))

    model.append(algo)
    precision.append(precision_score(y_test,y_pred))
    recall.append(recall_score(y_test,y_pred))
    F1score.append(f1_score(y_test,y_pred))
    AUCROC.append(roc_auc_score(y_test, y_prob[:,1]))
    resample.append(sampling)


## Model-1: Logistic Regression

### 1. `Original Unsampled Data`

In [ ]:
log_model=LogisticRegression()

params={'C':np.logspace(-10, 1, 15),'class_weight':[None,'balanced'],'penalty':['l1','l2']}

cv = StratifiedKFold(n_splits=5, random_state=100, shuffle=True)

# Create grid search using 5-fold cross validation
clf_LR = GridSearchCV(log_model, params, cv=cv, scoring='roc_auc', n_jobs=-1)
clf_LR.fit(X_train, y_train)
clf_LR.best_estimator_

In [ ]:
test_eval(clf_LR, X_test, y_test, 'Logistic Regression', 'actual')

### `2.SMOTE Resampling`

In [ ]:
clf_LR.fit(X_train_sm, y_train_sm)
clf_LR.best_estimator_

In [ ]:
test_eval(clf_LR, X_test, y_test, 'Logistic Regression', 'smote')

### `3.ADASYN Resampling`

In [ ]:
clf_LR.fit(X_train_ada, y_train_ada)
clf_LR.best_estimator_

In [ ]:
test_eval(clf_LR, X_test, y_test, 'Logistic Regression', 'adasyn')

### `4.SMOTE + Tomek Resampling`

In [ ]:
clf_LR.fit(X_train_smtom, y_train_smtom)
clf_LR.best_estimator_

In [ ]:
test_eval(clf_LR, X_test, y_test, 'Logistic Regression', 'smote+tomek')

### `5.SMOTE + ENN Resampling`

In [ ]:
clf_LR.fit(X_train_smenn, y_train_smenn)
clf_LR.best_estimator_

In [ ]:
test_eval(clf_LR, X_test, y_test, 'Logistic Regression', 'smote+enn')

## Model-2: Decision Tree

In [ ]:
estimators = [2,10,30,50,100]
# Maximum number of depth in each tree:
max_depth = [i for i in range(5,16,2)]
# Minimum number of samples to consider to split a node:
min_samples_split = [2, 5, 10, 15, 20, 50, 100]
# Minimum number of samples to consider at each leaf node:
min_samples_leaf = [1, 2, 5]

### `1. Original Unsampled Data`

In [ ]:
tree_model = DecisionTreeClassifier()

tree_param_grid = {
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf
}

clf_DT = RandomizedSearchCV(tree_model, tree_param_grid, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=2)
clf_DT.fit(X_train, y_train)
clf_DT.best_estimator_

In [ ]:
test_eval(clf_DT, X_test, y_test, 'Decision Tree', 'actual')

### `2.SMOTE Resampling`

In [ ]:
clf_DT.fit(X_train_sm, y_train_sm)
clf_DT.best_estimator_

In [ ]:
test_eval(clf_DT, X_test, y_test, 'Decision Tree', 'smote')

### `3.ADASYN Resampling`

In [ ]:
clf_DT.fit(X_train_ada, y_train_ada)
clf_DT.best_estimator_

In [ ]:
test_eval(clf_DT, X_test, y_test, 'Decision Tree', 'adasyn')

### `4. SMOTE + Tomek Resampling`

In [ ]:
clf_DT.fit(X_train_smtom, y_train_smtom)
clf_DT.best_estimator_

In [ ]:
test_eval(clf_DT, X_test, y_test, 'Decision Tree', 'smote+tomek')

### `5.SMOTE + ENN Resampling`

In [ ]:
clf_DT.fit(X_train_smenn, y_train_smenn)
clf_DT.best_estimator_

In [ ]:
test_eval(clf_DT, X_test, y_test, 'Decision Tree', 'smote+enn')

## Model-3: Random Forest

### 1. `Original Unsampled Data`

In [ ]:
rf_model = RandomForestClassifier()

rf_params={'n_estimators':estimators,
           'max_depth':max_depth,
           'min_samples_split':min_samples_split}

clf_RF = RandomizedSearchCV(rf_model, rf_params, cv=cv, scoring='roc_auc', n_jobs=-1, n_iter=20, verbose=2)
clf_RF.fit(X_train, y_train)
clf_RF.best_estimator_

In [ ]:
test_eval(clf_RF, X_test, y_test, 'Random Forest', 'actual')

### `2.SMOTE Resampling`

In [ ]:
clf_RF.fit(X_train_sm, y_train_sm)
clf_RF.best_estimator_

In [ ]:
test_eval(clf_RF, X_test, y_test, 'Random Forest', 'smote')

### `3.ADASYN Resampling`

In [ ]:
clf_RF.fit(X_train_ada, y_train_ada)
clf_RF.best_estimator_

In [ ]:
test_eval(clf_RF, X_test, y_test, 'Random Forest', 'adasyn')

### `4. SMOTE + Tomek Resampling`

In [ ]:
clf_RF.fit(X_train_smtom, y_train_smtom)
clf_RF.best_estimator_

In [ ]:
test_eval(clf_RF, X_test, y_test, 'Random Forest', 'smote+tomek')

### `5. SMOTE + ENN Resampling`

In [ ]:
clf_RF.fit(X_train_smenn, y_train_smenn)
clf_RF.best_estimator_

In [ ]:
test_eval(clf_RF, X_test, y_test, 'Random Forest', 'smote+enn')

## Model Comparision

In [ ]:
clf_eval_df = pd.DataFrame({'model':model,
                            'resample':resample,
                            'precision':precision,
                            'recall':recall,
                            'f1-score':F1score,
                            'AUC-ROC':AUCROC})

In [ ]:
clf_eval_df

In [ ]:
sns.set(font_scale=1.2)
#sns.palplot(sns.color_palette())
g = sns.FacetGrid(clf_eval_df, col="model", height=5)
g.map(sns.barplot, "resample", "recall", palette='twilight', order=["actual", "smote", "adasyn", "smote+tomek", "smote+enn"])
g.set_xticklabels(rotation=30)
g.set_xlabels(' ', fontsize=14)
g.set_ylabels('Recall', fontsize=14)

In [ ]:
# Adding a box plot to visualize the distribution of the 'age' variable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(8, 6))
sns.boxplot(x='churn', y='age', data=df)
plt.title('Box Plot of Age vs Churn')
plt.show()